In [1]:
import os
import tqdm
import wandb
import warnings
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import concurrent.futures

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow logging
warnings.filterwarnings('ignore', category=UserWarning, module='google.protobuf')

from matplotlib.axes import Axes
from wandb.apis.public import Run

from typing import Union, List, Dict
from src.visualization import set_themes

set_themes() # Set custom themes for plots
pl.Config.set_tbl_rows(20) # Set Polars table display rows limit

pd.set_option('future.no_silent_downcasting', True)

In [2]:
cache_file = "wandb/summary.parquet"
config = {
    "model": "matrix_factorization",
    "ensure_available_locally": False
}
sorting_criterion = {
    "epoch/test_hitrate@50": 0.5,
    "epoch/test_ndcg@50": 0.25,
}

if cache_file is not None and os.path.exists(cache_file):
    print(f"Loading cached experiment runs from {cache_file}...")
    experiment_runs = pl.read_parquet(cache_file)
    print(f"Loaded {len(experiment_runs)} runs from cache.")
else:
    print("No cache file found. Fetching experiment runs from Weights & Biases...")
    api = wandb.Api() # Initialize Weights & Biases API, used for fetching run data

    def fetch_run_metadata(run: Run, considered_metrics: Union[str, Dict[str, float]] = "epoch/epoch") -> Dict:
        run_config = {}
        for key, value in run.config.items():
            # Convert lists and dicts to strings
            if isinstance(value, (list, dict)):
                run_config[key] = str(value)
            else:
                run_config[key] = value

        run_history = run.history()
        run_history = run_history.replace({"Infinity": np.inf, "NaN": np.nan})

        if isinstance(considered_metrics, str):
            run_history["score"] = run_history[considered_metrics]
        elif isinstance(considered_metrics, dict):
            run_history["score"] = sum(
                run_history[metric] * weight for metric, weight in considered_metrics.items()
            )
        else:
            raise ValueError("considered_metrics must be either a string or a dictionary")
        
        best_summary = run_history.iloc[run_history["score"].argmax()]
        best_summary = {f"best:{key}": val for key, val in best_summary.items()}
        
        return {
            "run_id": run.id,
            "run_name": run.name,
            "sweep_id": run.sweep.id if run.sweep else None,
            "model": run.config.get("model"),
            **run_config,
            **{metric: run_history[metric].to_list() for metric in run_history},
            **best_summary,
            "gpu_type": run.metadata.get("gpu"),
            "cpu_count": run.metadata.get("cpu_count"),
        }

    batch_size = 16
    records = []
    futures = {}
    executor = concurrent.futures.ThreadPoolExecutor(max_workers=batch_size)
    runs:List[Run] = api.runs("feedr/peppermint-matrix", per_page=2*batch_size-1, filters={"config.model": config["model"]})
    run_iterator = iter(runs)
    with tqdm.tqdm(total=len(runs), ncols=128) as pbar:
        while len(records) < len(runs):
            # submit new tasks if we empty slots in the batch
            while len(futures) < batch_size and len(records) + len(futures) < len(runs):
                current_runs = next(run_iterator)
                current_future = executor.submit(fetch_run_metadata, current_runs, sorting_criterion)
                futures[current_future] = current_runs

            # check for completed tasks
            finished_futures, _ = concurrent.futures.wait(futures.keys(), return_when=concurrent.futures.FIRST_COMPLETED, timeout=0.1)
            for finished_future in finished_futures:
                finished_run = futures.pop(finished_future)
                records.append(finished_future.result())
                pbar.update(1)

    # Create a Polars DataFrame from the records
    experiment_runs = pl.DataFrame(records, infer_schema_length=None)
    
# Tag run as available locally if the model files exist
local_run_ids = []
local_sweep_ids = os.listdir(f"./models/{config['model']}/")
for sweep_id in local_sweep_ids:
    local_run_ids.extend([run_id for run_id in os.listdir(f"./models/{config['model']}/{sweep_id}/")])
    
experiment_runs = experiment_runs.with_columns(
    available_locally=pl.col("run_id").is_in(local_run_ids)
)

if config["ensure_available_locally"]:
    experiment_runs = experiment_runs.filter(pl.col("available_locally") == True)

experiment_runs = experiment_runs.sort("_timestamp", descending=False)
experiment_runs = experiment_runs.with_columns(
    run_duration_second=pl.col("_runtime").list.max(),
    run_duration_minute=(pl.col("_runtime").list.max() / 60)
)
experiment_runs.select(
    pl.col("run_id"),
    pl.col("run_name"),
    pl.col("sweep_id"),
    pl.col("model"),
    pl.col("embedding_dimension"),
    pl.col("shuffle"),
    pl.col("best:epoch/epoch"),
    pl.col("best:epoch/train_loss"),
    pl.col("best:epoch/test_loss"),
    pl.col("best:epoch/test_recall@10"),
    pl.col("best:epoch/test_ndcg@10"),
)

Loading cached experiment runs from wandb/summary.parquet...
Loaded 868 runs from cache.


run_id,run_name,sweep_id,model,embedding_dimension,shuffle,best:epoch/epoch,best:epoch/train_loss,best:epoch/test_loss,best:epoch/test_recall@10,best:epoch/test_ndcg@10
str,str,str,str,i64,bool,f64,f64,f64,f64,f64
"""o94q0juk""","""logical-sweep-1""","""nbysw136""","""matrix_factorization""",256,false,52.0,0.330564,0.372714,0.026077,0.100539
"""4ftaae0p""","""stilted-sweep-3""","""nbysw136""","""matrix_factorization""",4,false,59.0,0.693148,0.693148,0.006594,0.02556
"""fway5u2z""","""breezy-sweep-4""","""nbysw136""","""matrix_factorization""",512,false,4.0,0.693147,0.693147,0.0026,0.012059
"""bphcl2xf""","""clean-sweep-2""","""nbysw136""","""matrix_factorization""",1024,false,1.0,0.237272,0.239171,0.024744,0.095121
"""fftz1dek""","""trim-sweep-5""","""nbysw136""","""matrix_factorization""",256,true,57.0,0.319744,0.366027,0.025562,0.099834
"""otb8suw9""","""scarlet-sweep-6""","""nbysw136""","""matrix_factorization""",4,true,63.0,0.133806,0.217083,0.021467,0.082659
"""lvre7srl""","""solar-sweep-7""","""nbysw136""","""matrix_factorization""",256,true,8.0,0.036125,0.194509,0.021456,0.084617
"""dcbj92eg""","""ruby-sweep-8""","""nbysw136""","""matrix_factorization""",256,false,3.0,0.078173,0.179263,0.024295,0.094158
"""x17mnyw8""","""breezy-sweep-9""","""nbysw136""","""matrix_factorization""",8,false,61.0,0.111218,0.207902,0.023699,0.089898


# 28-01-2026 Adding Dropout
We need to make sure adding Dropout layer dont actually change the model in a negative way if dropout rate is set to 0.

In [3]:
experiment_runs_testcase1 = experiment_runs.filter(
    pl.col("shuffle") == True,
    pl.col("embedding_dimension") == 128,
    pl.col("l2_regularization") == 1e-7
)
experiment_runs_testcase1 = experiment_runs_testcase1.with_columns(
    test_group = pl.when(pl.col("created_at").dt.truncate("1d") <= pl.datetime(2026, 1, 27)).then(pl.lit("A")).otherwise(pl.lit("B")),
)
experiment_runs_testcase1

run_id,run_name,sweep_id,model,created_at,shuffle,log_freq,max_epoch,batch_size,random_seed,store_model,learning_rate,early_stopping,l1_regularization,l2_regularization,evaluation_cutoffs,early_stopping_mode,embedding_dimension,early_stopping_monitor,embedding_dropout_rate,early_stopping_patience,_timestamp,epoch/train_hitrate@50,epoch/train_precision@50,_step,epoch/train_ndcg@20,epoch/train_recall@50,epoch/test_mrr@50,epoch/train_hitrate@10,epoch/train_precision@20,epoch/test_precision@50,epoch/test_recall@50,epoch/train_hitrate@2,epoch/test_mrr@20,epoch/test_recall@10,epoch/train_mrr@2,epoch/train_mrr@10,…,best:epoch/test_map@50,best:epoch/test_mrr@10,best:epoch/epoch,best:epoch/test_hitrate@10,best:epoch/train_precision@2,best:epoch/test_hitrate@2,best:epoch/train_map@20,best:epoch/train_recall@2,best:epoch/test_precision@2,best:epoch/test_ndcg@20,best:epoch/test_hitrate@20,best:epoch/train_ndcg@50,best:epoch/test_recall@2,best:epoch/step_delta,best:epoch/train_hitrate@20,best:epoch/train_mrr@50,best:epoch/train_ndcg@10,best:epoch/train_ndcg@2,best:epoch/test_map@2,best:epoch/train_map@10,best:epoch/test_precision@10,best:epoch/train_mrr@20,best:epoch/train_map@50,best:epoch/test_loss,best:epoch/train_recall@10,best:epoch/train_map@2,best:_runtime,best:epoch/test_ndcg@2,best:epoch/train_loss,best:score,gpu_type,cpu_count,config,available_locally,run_duration_second,run_duration_minute,test_group
str,str,str,str,datetime[μs],bool,str,i64,i64,i64,bool,f64,bool,i64,f64,str,str,i64,str,i64,i64,list[f64],list[f64],list[f64],list[i64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,i64,str,bool,f64,f64,str
"""37n7u4k2""","""revived-sweep-62""","""nbysw136""","""matrix_factorization""",2026-01-24 19:24:05,true,"""epoch""",64,16384,1712,false,0.01,false,0,0.0000001,"""[2, 10, 20, 50]""","""max""",128,"""test_recall@10""",0,0,"[1.7693e9, 1.7693e9, … 1.7693e9]","[0.825218, 0.835575, … 0.998832]","[0.066459, 0.066809, … 0.201024]","[0, 1, … 63]","[0.345739, 0.332062, … 0.657331]","[0.095013, 0.096401, … 0.309321]","[0.068349, 0.066969, … 0.088]","[0.534862, 0.51942, … 0.941171]","[0.087006, 0.085323, … 0.261268]","[0.012879, 0.013, … 0.017378]","[0.064807, 0.068126, … 0.095012]","[0.244505, 0.215991, … 0.625774]","[0.063729, 0.062122, … 0.081941]","[0.019571, 0.020122, … 0.027622]","[0.197076, 0.175398, … 0.536204]","[0.257642, 0.238751, … 0.61007]",…,0.070885,0.07405,55.0,0.205033,0.40762,0.06025,0.447995,0.027943,0.031088,0.132567,0.322881,0.640787,0.006923,8459.0,0.985064,0.614196,0.661914,0.560315,0.046372,0.516281,0.024507,0.613698,0.35882,0.250032,0.099302,0.537104,721.745169,0.050006,0.094676,0.194582,"""NVIDIA A10G""",8,null,false,812.153459,13.535891,"""A"""
"""eiy3ub63""","""bright-sweep-253""","""nbysw136""","""matrix_factorization""",2026-01-25 05:23:54,true,"""epoch""",64,16384,11769,false,0.01,false,0,0.0000001,"""[2, 10, 20, 50]""","""max""",128,"""test_recall@10""",0,0,"[1.7693e9, 1.7693e9, … 1.7693e9]","[0.813913, 0.826007, … 0.998895]","[0.064144, 0.064884, … 0.201906]","[0, 1, … 63]","[0.3363, 0.326247, … 0.660678]","[0.091446, 0.093368, … 0.310944]","[0.066747, 0.064099, … 0.089749]","[0.52602, 0.507294, … 0.943729]","[0.083316, 0.081903, … 0.263327]","[0.012542, 0.012573, … 0.017515]","[0.063228, 0.065425, … 0.095369]","[0.24419, 0.220443, … 0.634521]","[0.062224, 0.059297, … 0.083623]","[0.019471, 0.019138, … 0.027776]","[0.192687, 0.180577, … 0.542598]","[0.251299, 0.237904, … 0.61544]",…,0.072625,0.076987,60.0,0.210591,0.40942,0.062334,0.450859,0.028043,0.032193,0.13515,0.325755,0.644234,0.007174,8459.0,0.985822,0.616209,0.665734,0.561435,0.048582,0.519373,0.025382,0.615737,0.362817,0.24999,0.101436,0.538083,715.856493,0.052183,0.094722,0.196665,"""NVIDIA A10G""",8,null,false,743.344468,12.389074,"""A"""
"""iz9ur6ru"

In [5]:
experiment_runs_testcase1.group_by("test_group").agg(
    pl.col("run_id").count().alias("num_runs"),
    pl.col("best:epoch/epoch").mean(),
    pl.col("best:epoch/test_recall@10").mean(),
    pl.col("best:epoch/test_ndcg@10").mean(),
    pl.col("best:epoch/test_recall@20").mean(),
    pl.col("best:epoch/test_ndcg@20").mean(),
).to_pandas().transpose()

,0,1
test_group,A,B
num_runs,5,5
best:epoch/epoch,57.0,58.6
best:epoch/test_recall@10,0.028025,0.028115
best:epoch/test_ndcg@10,0.10642,0.105992
best:epoch/test_recall@20,0.048683,0.048705
best:epoch/test_ndcg@20,0.134515,0.13399
